In [1]:
import pandas as pd 
import torch 
import torch.nn as nn 
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import numpy as np 
import argparse
from copy import deepcopy #Add Deepcopy for args
import visdom
vis = visdom.Visdom()

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

print(torch.__version__)
%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (15, 9)

Setting up a new session...


1.4.0
Populating the interactive namespace from numpy and matplotlib


# 1. Data loading & Preprocessing

### 1.1 Data loading

In [2]:
cd /Users/jinsungpark/Desktop/Data_river/data04

[Errno 2] No such file or directory: '/Users/jinsungpark/Desktop/Data_river/data04'
/Users/jinsungpark/Desktop/jupyter


In [3]:
pwd #현재 경로 확인

'/Users/jinsungpark/Desktop/jupyter'

In [4]:
ls #현재경로에 있는 항목 확인

Correlation_analysis.ipynb
Data_Preprocessing.ipynb
Data_river/
Lab10_Stock_Price_Prediction_with_LSTM.ipynb
Pandas_tutorial.ipynb
Project_Git/
Project_River_Backup.ipynb
Project_River_Backup_change_TP.ipynb
Project_River_Find_BestModel.ipynb
Project_River_Find_BestModel_01.ipynb
Project_River_matplotlib.ipynb
Project_River_matplotlib_01.ipynb
Untitled.ipynb
Untitled1.ipynb
results/


In [5]:
cd /Users/jinsungpark/Desktop/jupyter/Data_river/data04

/Users/jinsungpark/Desktop/jupyter/Data_river/data04


In [15]:
UpStream_data = pd.read_excel('DS_Data_edit_log.xlsx')
DownStream_data = pd.read_excel('NG_Data_edit_log.xlsx')

In [7]:
print(UpStream_data.columns)
# print(DownStream_data.columns)

Index(['Date', 'DS_DO', 'DS_BOD', 'DS_COD', 'DS_SS', 'DS_TN', 'DS_TP',
       'DS_Chl_a', 'DS_Cells', 'GJ_Deep', 'GJ_Level', 'GJ_Outflow',
       'DaeGu_Rain', 'DaeGu_Solar', 'SeoBu_COD', 'SeoBu_SS', 'SeoBu_TN',
       'SeoBu_TP', 'SeoBu_Flow_mean', 'SeoBu_Flow_day', 'SeoBu_COD_load',
       'SeoBu_SS_load', 'SeoBu_TN_load', 'SeoBu_TP_load', 'SungSeo_COD',
       'SungSeo_SS', 'SungSeo_TN', 'SungSeo_TP', 'SungSeo_Flow_mean',
       'SungSeo_Flow_day', 'SungSeo_COD_load', 'SungSeo_SS_load',
       'SungSeo_TN_load', 'SungSeo_TP_load', 'GumHo_DO', 'GumHo_BOD',
       'GumHo_COD', 'GumHo_SS', 'GumHo_TN', 'GumHo_TP', 'GumHo_Chl_a',
       'GumHo_Flow', 'DS_Temp', 'GumHo_Temp'],
      dtype='object')


In [16]:
#날짜 인덱스화
UpData = UpStream_data.set_index('Date')
DownData = DownStream_data.set_index('Date')

In [17]:
UpData.info()

<class 'pandas.core.frame.DataFrame'>
Index: 349 entries, 2013-01-07 to 2019-09-30
Data columns (total 43 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   DS_DO              349 non-null    float64
 1   DS_BOD             349 non-null    float64
 2   DS_COD             349 non-null    float64
 3   DS_SS              349 non-null    float64
 4   DS_TN              349 non-null    float64
 5   DS_TP              349 non-null    float64
 6   DS_Chl_a           349 non-null    float64
 7   DS_Cells           349 non-null    float64
 8   GJ_Deep            349 non-null    float64
 9   GJ_Level           349 non-null    float64
 10  GJ_Outflow         349 non-null    float64
 11  DaeGu_Rain         349 non-null    float64
 12  DaeGu_Solar        349 non-null    float64
 13  SeoBu_COD          349 non-null    float64
 14  SeoBu_SS           349 non-null    float64
 15  SeoBu_TN           349 non-null    float64
 16  SeoBu_TP       

In [21]:
#넣고싶은 상류 항목 컬럼 선택
UpData = UpData.iloc[:,[0,4,7,10,11,12,15,33,35,37,38,39,41,42]]
UpData.info()

<class 'pandas.core.frame.DataFrame'>
Index: 349 entries, 2013-01-07 to 2019-09-30
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   DS_DO        349 non-null    float64
 1   DS_TN        349 non-null    float64
 2   DS_Cells     349 non-null    float64
 3   GJ_Outflow   349 non-null    float64
 4   DaeGu_Rain   349 non-null    float64
 5   DaeGu_Solar  349 non-null    float64
 6   SeoBu_TN     349 non-null    float64
 7   GumHo_DO     349 non-null    float64
 8   GumHo_COD    349 non-null    float64
 9   GumHo_TN     349 non-null    float64
 10  GumHo_TP     349 non-null    float64
 11  GumHo_Chl_a  349 non-null    float64
 12  DS_Temp      349 non-null    float64
 13  GumHo_Temp   349 non-null    float64
dtypes: float64(14)
memory usage: 40.9+ KB


In [19]:
DownData.info()

<class 'pandas.core.frame.DataFrame'>
Index: 349 entries, 2013-01-07 to 2019-09-30
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   NG_DO     349 non-null    float64
 1   NG_BOD    349 non-null    float64
 2   NG_COD    349 non-null    float64
 3   NG_SS     349 non-null    float64
 4   NG_TN     349 non-null    float64
 5   NG_TP     349 non-null    float64
 6   NG_Chl_a  349 non-null    float64
dtypes: float64(7)
memory usage: 21.8+ KB


In [20]:
#알고싶은 하류 항목 컬럼 넘버 넣기('Date'항목이 인덱스화 돼서 컬럼 넘버가 -1씩 됨)
Colum = 4
print(DownData.columns[Colum])

NG_TN


### 1.2 Data Preprocessing(normalization)

In [22]:
from sklearn.preprocessing import MinMaxScaler

UpScaler = MinMaxScaler() #상류데이터용
DownScaler = MinMaxScaler() #하류데이터용

#나중에 결과를 DeNormalizing 하기 위해 나누어 사용 하였다.

def DeNormalize(Y, Data_name, column_num, Scaler_Type):
    
    data = Data_name
    Scaler = Scaler_Type
    
    _max = Scaler.data_max_[column_num] # 역정규화 하려는 데이터의 컬럼 번호
    _min = Scaler.data_min_[column_num] 
    
    X = Y*(_max-_min) + _min
    
    return X

In [23]:
#데이터 정규화
UpData = pd.DataFrame(UpScaler.fit_transform(UpData), columns=UpData.columns, index=UpData.index)
DownData = pd.DataFrame(DownScaler.fit_transform(DownData), columns=DownData.columns, index=DownData.index)

print(UpData.isna().sum())

DS_DO          0
DS_TN          0
DS_Cells       0
GJ_Outflow     0
DaeGu_Rain     0
DaeGu_Solar    0
SeoBu_TN       0
GumHo_DO       0
GumHo_COD      0
GumHo_TN       0
GumHo_TP       0
GumHo_Chl_a    0
DS_Temp        0
GumHo_Temp     0
dtype: int64


#2. Data Preparation

In [24]:
class RiverDataset(Dataset):
    def __init__(self, UpData, DownData, x_frames, y_frames, start, end):
        
        self.x_frames = x_frames
        self.y_frames = y_frames
        
        self.start = start
        self.end = end

        self.UpData = UpData[start:end]
        self.DownData = DownData[start:end]

    def __len__(self):
        return len(self.UpData) - (self.x_frames + self.y_frames) + 1
    #데이터를 전처리 할때 UpData와 DownData의 길이가 동일해짐(날짜를 동일한것만 추출해야 하므로), 따라서 전체길이는 둘중 하나를 사용

    def __getitem__(self, idx):
        idx += self.x_frames

        X = self.UpData.iloc[idx-self.x_frames:idx].values
        Y = self.DownData.iloc[idx:idx+self.y_frames].values
        
        return X, Y

# 3. Model Define

In [25]:
class LSTM(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, batch_size, dropout, use_bn):
        super(LSTM, self).__init__()
        self.input_dim = input_dim 
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers

        self.batch_size = batch_size
        self.dropout = dropout
        self.use_bn = use_bn 
        
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers) #
        self.hidden = self.init_hidden()
        self.regressor = self.make_regressor()
        
    def init_hidden(self):
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))
    
    def make_regressor(self):
        layers = []
        if self.use_bn:
            layers.append(nn.BatchNorm1d(self.hidden_dim))
        layers.append(nn.Dropout(self.dropout))
        
        layers.append(nn.Linear(self.hidden_dim, self.hidden_dim // 2))
        layers.append(nn.ReLU())
        layers.append(nn.Linear(self.hidden_dim // 2, self.output_dim))
        regressor = nn.Sequential(*layers)
        return regressor
    
    def forward(self, x):
        lstm_out, self.hidden = self.lstm(x, self.hidden)
        y_pred = self.regressor(lstm_out[-1].view(self.batch_size, -1))
        return y_pred

In [26]:
# 정확도 : 예측확률을 100%로 봤을때 MAPE에 따른 오차비율을 빼줌 (100-MAPE) ##RMSE, MAPE 두개로 볼 수 있게
def MAPE(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [27]:
def RMSE(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred, multioutput='raw_values')
    return np.sqrt(mse)

In [28]:
def R2(y_true, y_pred):
    R2_score = r2_score(y_true, y_pred, multioutput='raw_values')
    return R2_score

# 4. Train, Validate, Test

In [29]:
def train(model, partition, optimizer, loss_fn, args):
    trainloader = DataLoader(partition['train'],
                             batch_size=args.batch_size,
                             shuffle=False, drop_last=True)
    model.train()
    model.zero_grad()
    optimizer.zero_grad()

    bat_siz = args.batch_size
    pred = []
    true = []
    pred_results = []
    true_results = []
    train_acc = 0.0
    train_loss = 0.0
    
    for i, (X, y) in enumerate(trainloader):

        X = X.transpose(0, 1).float().to(args.device)#파이토치는 순서가 달라서 바꿔줌
        y_true = y[:, :, Colum].float().to(args.device)
        model.zero_grad()
        optimizer.zero_grad()
        model.hidden = [hidden.to(args.device) for hidden in model.init_hidden()]

        y_pred = model(X)
        loss = loss_fn(y_pred.view(-1), y_true.view(-1))
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        pred.append(y_pred)
        true.append(y_true)

    # ==== test 데이터 시각화를 위해 x,y데이터 저장 (배치사이즈의 텐서들을 한개씩 추가해주기) ==== #
    for i in range(len(trainloader)):
        tems1 = pred[i].view(bat_siz).cpu().detach().numpy()
        tems2 = true[i].view(bat_siz).cpu().detach().numpy()
        
        for j in range(bat_siz):
            value1 = np.exp(DeNormalize(tems1[j], DownData, Colum, DownScaler))
            value2 = np.exp(DeNormalize(tems2[j], DownData, Colum, DownScaler))
            
            pred_results.append(value1)
            true_results.append(value2)
    # ========================================================================== #   

    train_loss = train_loss / len(trainloader)
    train_acc1 = RMSE(np.array(true_results), np.array(pred_results))
    train_acc2 = R2(np.array(true_results), np.array(pred_results))
#     train_acc3 = (100 - MAPE(np.array(true_results), np.array(pred_results)))

    return model, train_loss, train_acc1[0], train_acc2[0]

In [30]:
def validate(model, partition, loss_fn, args):
    valloader = DataLoader(partition['val'],
                           batch_size=args.batch_size,
                           shuffle=False, drop_last=True)
    model.eval()

    bat_siz = args.batch_size
    pred = []
    true = []
    pred_results = []
    true_results = []
    val_acc = 0.0
    val_loss = 0.0
    
    with torch.no_grad():
        for i, (X, y) in enumerate(valloader):

            X = X.transpose(0, 1).float().to(args.device)
            y_true = y[:, :, Colum].float().to(args.device)
            model.hidden = [hidden.to(args.device) for hidden in model.init_hidden()]

            y_pred = model(X)
            loss = loss_fn(y_pred.view(-1), y_true.view(-1))

            val_loss += loss.item()

            pred.append(y_pred)
            true.append(y_true)

    # ==== test 데이터 시각화를 위해 x,y데이터 저장 (배치사이즈의 텐서들을 한개씩 추가해주기) ==== #
    for i in range(len(valloader)):
        tems1 = pred[i].view(bat_siz).cpu().detach().numpy()
        tems2 = true[i].view(bat_siz).cpu().detach().numpy()
        
        for j in range(bat_siz):
            value1 = np.exp(DeNormalize(tems1[j], DownData, Colum, DownScaler))
            value2 = np.exp(DeNormalize(tems2[j], DownData, Colum, DownScaler))
            pred_results.append(value1)
            true_results.append(value2)
    # ========================================================================== #   
    
    val_loss = val_loss / len(valloader)
    val_acc1 = RMSE(np.array(true_results), np.array(pred_results))
    val_acc2 = R2(np.array(true_results), np.array(pred_results))
#     val_acc3 = (100 - MAPE(np.array(true_results), np.array(pred_results)))

    
    return val_loss, val_acc1[0], val_acc2[0]

In [31]:
def test(model, partition, args):
    testloader = DataLoader(partition['test'],
                           batch_size=args.batch_size,
                           shuffle=False, drop_last=True)
    model.eval()

    bat_siz = args.batch_size
    pred = []
    true = []
    pred_results = []
    true_results = []
    test_acc = 0.0
    
    with torch.no_grad():
        for i, (X, y) in enumerate(testloader):
            X = X.transpose(0, 1).float().to(args.device)
            y_true = y[:, :, Colum].float().to(args.device)
            model.hidden = [hidden.to(args.device) for hidden in model.init_hidden()]

            y_pred = model(X)

            pred.append(y_pred)
            true.append(y_true)

    # ==== test 데이터 시각화를 위해 x,y데이터 저장 ==== #
    for i in range(len(testloader)):
        tems1 = pred[i].view(bat_siz).cpu().detach().numpy()
        tems2 = true[i].view(bat_siz).cpu().detach().numpy()
        
        for j in range(bat_siz):
            value1 = np.exp(DeNormalize(tems1[j], DownData, Colum, DownScaler))
            value2 = np.exp(DeNormalize(tems2[j], DownData, Colum, DownScaler))
            
            pred_results.append(value1)
            true_results.append(value2)
    # ========================================== #   

    test_acc1 =  RMSE(np.array( true_results), np.array(pred_results))
    test_acc2 =  R2(np.array( true_results), np.array(pred_results))
#     test_acc3 =  (100 - MAPE(np.array( true_results), np.array(pred_results)))
    
    return test_acc1[0], test_acc2[0], pred_results, true_results

# 5. Experiment

In [32]:
def experiment(partition, args):

    model = LSTM(args.input_dim, args.hid_dim, args.y_frames, args.n_layers, args.batch_size, args.dropout, args.use_bn)
    model.to(args.device)
#     loss_fn = torch.nn.MSELoss() ##loss는 mse를 사용
#     loss_fn = nn.MSELoss()
    
    if args.loss == 'MSELoss':
        loss_fn = torch.nn.MSELoss()
        loss_fn = nn.MSELoss()
    elif args.loss == 'L1Loss':
        loss_fn = torch.nn.L1Loss()
        loss_fn = nn.L1Loss()
    elif args.loss == 'PoissonNLLLoss':
        loss_fn = torch.nn.PoissonNLLLoss()
        loss_fn = nn.PoissonNLLLoss()
    elif args.loss == 'KLDivLoss':
        loss_fn = torch.nn.KLDivLoss()
        loss_fn = nn.KLDivLoss()
    elif args.loss == 'BCELoss':
        loss_fn = torch.nn.BCELoss()
        loss_fn = nn.BCELoss()
    elif args.loss == 'BCEWithLogitsLoss':
        loss_fn = torch.nn.BCEWithLogitsLoss()
        loss_fn = nn.BCEWithLogitsLoss()
    else:
        raise ValueError('In-valid LossFuction choice')
    
    
    
    if args.optim == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=args.lr, weight_decay=args.l2)
    elif args.optim == 'RMSprop':
        optimizer = optim.RMSprop(model.parameters(), lr=args.lr, weight_decay=args.l2)
    elif args.optim == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.l2)
    else:
        raise ValueError('In-valid optimizer choice')
    
    # ===== List for epoch-wise data ====== #
    train_losses = []
    val_losses = []
    train_accs_RMSE = []
    train_accs_R2 = []
    val_accs_RMSE = []
    val_accs_R2 = []
    axis = []
    # ===================================== #
    
    ## model starting point ##    
    ts = time.time()
    model, train_loss, train_acc_RMSE, train_acc_R2 = train(model, partition, optimizer, loss_fn, args)
    val_loss, val_acc_RMSE, val_acc_R2 = validate(model, partition, loss_fn, args)
    te = time.time()

    # ====== Add Epoch Data ====== #
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accs_RMSE.append(train_acc_RMSE)
    val_accs_RMSE.append(val_acc_RMSE)
    train_accs_R2.append(train_acc_R2)
    val_accs_R2.append(val_acc_R2)
    # ============================ #

    # # ===== Visdom visualizing ================================================================================== #
    axis.append(0)
    
    plot1 = vis.line(Y=torch.cat((torch.Tensor(train_losses).view(-1,1), torch.Tensor(val_losses).view(-1,1)), -1),
                     X=torch.cat((torch.Tensor(axis).view(-1,1), torch.Tensor(axis).view(-1,1)), -1),
                     opts=dict(title='exp_{}_loss'.format(num), legend=['train_loss','val_loss'], showlegend=True))
    
    plot2 = vis.line(Y=torch.cat((torch.Tensor(train_accs_RMSE).view(-1,1), torch.Tensor(val_accs_RMSE).view(-1,1)), -1),
                     X=torch.cat((torch.Tensor(axis).view(-1,1), torch.Tensor(axis).view(-1,1)), -1),
                     opts=dict(title='exp_{}_acc_RMSE'.format(num), legend=['train_acc','val_acc'], showlegend=True))
    # # =========================================================================================================== #
    
    print('Epoch {}, Acc_RMSE(train/val): {:2.2f}/{:2.2f}, Loss(train/val) {:2.5f}/{:2.5f}. Took {:2.2f} sec'
          .format(0, train_acc_RMSE, val_acc_RMSE, train_loss, val_loss, te-ts))
    
    for epoch in range(args.epoch-1):  # loop over the dataset multiple times
        
        ts = time.time()
        model, train_loss, train_acc_RMSE, train_acc_R2 = train(model, partition, optimizer, loss_fn, args)
        val_loss, val_acc_RMSE, val_acc_R2 = validate(model, partition, loss_fn, args)
        te = time.time()

        # ====== Add Epoch Data ====== #
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accs_RMSE.append(train_acc_RMSE)
        val_accs_RMSE.append(val_acc_RMSE)
        train_accs_R2.append(train_acc_R2)
        val_accs_R2.append(val_acc_R2)
        # ============================ #

        # # ===== Visdom visualizing ============================================================================== #
        axis.append(epoch+1)
        
        vis.line(Y=torch.cat((torch.Tensor(train_losses).view(-1,1), torch.Tensor(val_losses).view(-1,1)), -1),
                 X=torch.cat((torch.Tensor(axis).view(-1,1), torch.Tensor(axis).view(-1,1)), -1),
                 win=plot1, update='replace')
        
        vis.line(Y=torch.cat((torch.Tensor(train_accs_RMSE).view(-1,1), torch.Tensor(val_accs_RMSE).view(-1,1)), -1),
                 X=torch.cat((torch.Tensor(axis).view(-1,1), torch.Tensor(axis).view(-1,1)), -1),
                 win=plot2, update='replace')
        # # ====================================================================================================== #
        
        print('Epoch {}, Acc_RMSE(train/val): {:2.2f}/{:2.2f}, Loss(train/val) {:2.5f}/{:2.5f}. Took {:2.2f} sec'
              .format(epoch+1, train_acc_RMSE, val_acc_RMSE, train_loss, val_loss, te-ts))
        
    test_acc_RMSE, test_acc_R2, Pred_data, True_data = test(model, partition, args)
    
    # ======= Add Result to Dictionary ======= #
    result = {}
    result['train_losses'] = train_losses
    result['val_losses'] = val_losses
    
    result['train_accs_RMSE'] = train_accs_RMSE
    result['train_accs_R2'] = train_accs_R2
    result['val_accs_RMSE'] = val_accs_RMSE
    result['val_accs_R2'] = val_accs_R2
#     result['train_acc'] = train_acc
#     result['val_acc'] = val_acc
    result['test_RMSE'] = test_acc_RMSE
    result['test_R2'] = test_acc_R2
    
    result['test_pred'] = Pred_data
    result['test_true'] = True_data
    return result

# 6. LSTM Run

In [49]:
# ====== Random Seed Initialization ====== #
seed = 666
np.random.seed(seed)
torch.manual_seed(seed)

parser = argparse.ArgumentParser()
args = parser.parse_args("")

args.device = 'cuda' if torch.cuda.is_available() else 'cpu'

# ====== Data Loading ====== #
args.batch_size = 8 # 2~8
args.UpData = UpData
args.DownData = DownData
args.x_frames = 4
args.y_frames = 1

# ====== Model Capacity ===== #
args.input_dim = len(UpData.columns)
args.hid_dim = 16
args.n_layers = 3

# ====== Regularization ======= #
args.l2 = 0.0001 
args.dropout = 0.1 
args.use_bn = False

# ====== Optimizer & Training ====== #
args.optim = 'Adam' #SGD, RMSprop, Adam...
args.loss = 'MSELoss'#'MSELoss','L1Loss','PoissonNLLLoss','KLDivLoss','BCELoss','BCEWithLogitsLoss'
args.lr = 0.01
args.epoch = 180


# ====== Experiment Variable ====== #
name_var1 = 'x_frames'
name_var2 = 'batch_size'
name_var3 = 'optim'
list_var1 = [2, 3, 4]
list_var2 = [2, 4, 8]
list_var3 = ['Adam','RMSprop']
num = 0 #초기화


trainset = RiverDataset(args.UpData, args.DownData, args.x_frames, args.y_frames, '2013-01-01', '2016-07-31')
valset = RiverDataset(args.UpData, args.DownData, args.x_frames, args.y_frames, '2016-08-01', '2017-05-19')
testset = RiverDataset(args.UpData, args.DownData, args.x_frames, args.y_frames, '2016-08-01', '2017-05-19')
partition = {'train': trainset, 'val':valset, 'test':testset}


print('size of trainset :{}'.format(len(trainset)),
      'size of valset :{}'.format(len(valset)),
      'size of testset :{}'.format(len(testset)))

for var1 in list_var1:
    for var2 in list_var2:
        for var3 in list_var3:
            num += 1
            setattr(args, name_var1, var1)
            setattr(args, name_var2, var2)
            setattr(args, name_var3, var3)
            print('\n exp_{}'.format(num))
            print('loseFunc = {}, optim={}, x_frames={}, n_layers={}, batch_size={}, hid_dim={}, epoch={}, lr={}, l2={}, dropout={}, use_bn={}'
                  .format(args.loss,args.optim,args.x_frames,args.n_layers,args.batch_size,args.hid_dim,args.epoch,args.lr,args.l2,args.dropout,args.use_bn))     

            result = experiment(partition, deepcopy(args))

    #         print('train_acc_RMSE = {:2.2f}%, val_acc_RMSE = {:2.2f}%, test_RMSE = {:2.2f}%, test_R2 = {:2.2f}%'
    #               .format(result['train_acc_RMSE'],result['val_acc_RMSE'],result['test_RMSE'],result['test_R2']))

            vis.text('loseFunc = {}, optim={}, x_frames={}, n_layers={}, batch_size={}, hid_dim={}, epoch={}, lr={}, l2={}, dropout={}, use_bn={}'
                     .format(args.loss,args.optim,args.x_frames,args.n_layers,args.batch_size,args.hid_dim,args.epoch,args.lr,args.l2,args.dropout,args.use_bn),
                     opts=dict(title='exp_{}_text'.format(num)))
            # 만든 모델의 test 데이터 예측 시각화

            predict = torch.Tensor(result['test_pred']).view(-1,1)
            truth = torch.Tensor(result['test_true']).view(-1,1)
            axis = torch.Tensor(range(len(result['test_pred']))).view(-1,1)

            Y_axis = torch.cat((predict, truth), -1)
            X_axis = torch.cat((axis, axis), -1)

            vis.line(Y = Y_axis, X = X_axis, opts=dict(title='Result_exp_{}_RMSE[{:2.3f}]_R2[{:2.3f}]'.format(num,result['test_RMSE'],result['test_R2']),
                                                       legend=['predict','true'],
                                                       showlegend=True,
                                                       layoutopts = {'plotly': {'legend': {'x':0, 'y':0}}}))

print('All done')

size of trainset :181 size of valset :38 size of testset :38

 exp_1
loseFunc = MSELoss, optim=Adam, x_frames=2, n_layers=3, batch_size=2, hid_dim=16, epoch=10, lr=0.01, l2=0.0001, dropout=0.1, use_bn=False
Epoch 0, Acc_RMSE(train/val): 1.58/0.52, Loss(train/val) 0.09125/0.00970. Took 0.64 sec
Epoch 1, Acc_RMSE(train/val): 1.23/0.51, Loss(train/val) 0.04599/0.00930. Took 0.65 sec
Epoch 2, Acc_RMSE(train/val): 1.22/0.49, Loss(train/val) 0.04512/0.00856. Took 0.65 sec
Epoch 3, Acc_RMSE(train/val): 1.17/0.47, Loss(train/val) 0.04195/0.00810. Took 0.64 sec
Epoch 4, Acc_RMSE(train/val): 1.17/0.49, Loss(train/val) 0.04066/0.00860. Took 0.65 sec
Epoch 5, Acc_RMSE(train/val): 1.14/0.48, Loss(train/val) 0.03911/0.00777. Took 0.64 sec
Epoch 6, Acc_RMSE(train/val): 0.98/0.94, Loss(train/val) 0.02756/0.03382. Took 0.64 sec
Epoch 7, Acc_RMSE(train/val): 1.17/0.50, Loss(train/val) 0.04118/0.00867. Took 0.64 sec
Epoch 8, Acc_RMSE(train/val): 1.10/0.51, Loss(train/val) 0.03634/0.00881. Took 0.63 sec
E

Epoch 9, Acc_RMSE(train/val): 0.59/0.45, Loss(train/val) 0.01125/0.00683. Took 0.61 sec

 exp_9
loseFunc = MSELoss, optim=Adam, x_frames=3, n_layers=3, batch_size=4, hid_dim=16, epoch=10, lr=0.01, l2=0.0001, dropout=0.1, use_bn=False
Epoch 0, Acc_RMSE(train/val): 1.51/0.57, Loss(train/val) 0.07668/0.01067. Took 0.36 sec
Epoch 1, Acc_RMSE(train/val): 1.15/0.56, Loss(train/val) 0.04035/0.01022. Took 0.36 sec
Epoch 2, Acc_RMSE(train/val): 1.14/0.46, Loss(train/val) 0.03936/0.00715. Took 0.35 sec
Epoch 3, Acc_RMSE(train/val): 1.07/0.36, Loss(train/val) 0.03467/0.00511. Took 0.35 sec
Epoch 4, Acc_RMSE(train/val): 0.87/0.90, Loss(train/val) 0.02273/0.02129. Took 0.36 sec
Epoch 5, Acc_RMSE(train/val): 0.87/0.35, Loss(train/val) 0.02041/0.00414. Took 0.36 sec
Epoch 6, Acc_RMSE(train/val): 0.70/0.40, Loss(train/val) 0.01405/0.00502. Took 0.35 sec
Epoch 7, Acc_RMSE(train/val): 0.74/0.32, Loss(train/val) 0.01499/0.00350. Took 0.35 sec
Epoch 8, Acc_RMSE(train/val): 0.79/0.28, Loss(train/val) 0.016

Epoch 9, Acc_RMSE(train/val): 0.59/0.45, Loss(train/val) 0.01145/0.00772. Took 0.34 sec

 exp_17
loseFunc = MSELoss, optim=Adam, x_frames=4, n_layers=3, batch_size=8, hid_dim=16, epoch=10, lr=0.01, l2=0.0001, dropout=0.1, use_bn=False
Epoch 0, Acc_RMSE(train/val): 1.56/0.98, Loss(train/val) 0.07649/0.03674. Took 0.20 sec
Epoch 1, Acc_RMSE(train/val): 1.19/0.50, Loss(train/val) 0.04342/0.00821. Took 0.21 sec
Epoch 2, Acc_RMSE(train/val): 1.08/0.49, Loss(train/val) 0.03440/0.00795. Took 0.20 sec
Epoch 3, Acc_RMSE(train/val): 1.03/0.35, Loss(train/val) 0.03082/0.00380. Took 0.20 sec
Epoch 4, Acc_RMSE(train/val): 0.85/0.30, Loss(train/val) 0.01930/0.00316. Took 0.20 sec
Epoch 5, Acc_RMSE(train/val): 0.78/0.48, Loss(train/val) 0.01680/0.00875. Took 0.21 sec
Epoch 6, Acc_RMSE(train/val): 0.84/0.47, Loss(train/val) 0.01906/0.00781. Took 0.20 sec
Epoch 7, Acc_RMSE(train/val): 0.85/0.36, Loss(train/val) 0.01993/0.00526. Took 0.22 sec
Epoch 8, Acc_RMSE(train/val): 0.82/0.29, Loss(train/val) 0.01